In [1]:
from load_results import load_result_dataset
import pandas as pd

pn1 = 'full_fine_tuning_50epochs_edge_paper_final2'
pn2 = 'full_fine_tuning_50epochs_paper_final2'
final_data1 = load_result_dataset(pn1, pn2)
df1 = pd.DataFrame(final_data1)
df1['ft_strategy'] = 'full_fine_tuning_50epochs'

pn1 = 'full_fine_tuning_5epochs_edge_article1'
pn2 = 'full_fine_tuning_5epochs_article1'
final_data2 = load_result_dataset(pn1, pn2)
df2 = pd.DataFrame(final_data2)
df2['ft_strategy'] = 'full_fine_tuning_5epochs'

pn1 = 'linearprobe_50epochs_edge_paper_final2'
pn2 = 'linearprobe_50epochs_paper_final2'
final_data3 = load_result_dataset(pn1, pn2)
df3 = pd.DataFrame(final_data3)
df3['ft_strategy'] = 'linearprobe_50epochs'

df = pd.concat([ df1,df2,df3 ], axis=0, ignore_index=True) 

../results/full_fine_tuning_50epochs_paper_final2/CLIP-convnext_base_w-laion_aesthetic-s13B-b82K_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_paper_final2/CLIP-convnext_base_w-laion2B-s13B-b82K_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_paper_final2/deit_small_patch16_224.fb_in1k_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_paper_final2/robust_resnet50_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_paper_final2/vit_small_patch16_224.augreg_in21k_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_paper_final2/convnext_base.fb_in1k_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_paper_final2/resnet50.a1_in1k_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_paper_final2/robust_vit_base_patch16_224_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_

In [ ]:
# df = df[ df.model_size == 2 ]



Percentage of NaN values: 10.67%


(240, 48)

In [ ]:



df_sorted

dataset                                                                                               caltech101  \
metric                                                                                                    L1_acc   
ft_strategy               model_type          loss_function backbone                                               
full_fine_tuning_50epochs fully convolutional TRADES_v2     convnext_tiny.fb_in1k                       0.822000   
                          fully attention     TRADES_v2     vit_base_patch16_clip_224.laion2b_ft_in1k   0.776000   
                          fully convolutional TRADES_v2     convnext_tiny.fb_in22k                      0.805000   
                                                            robust_convnext_tiny                        0.586667   
                                                            edgenext_small.usi_in1k                     0.794000   
...                                                                                                          ...   
full_fine_tuning_5epochs  hybrid              CLASSIC_AT    mobilevit-small                                  NaN   
linearprobe_50epochs      fully convolutional CLASSIC_AT    edgenext_small.usi_in1k                          NaN   
                                                            efficientnet-b0                                  NaN   
                                                            mobilenetv3_large_100.ra_in1k                    NaN   
                          hybrid              TRADES_v2     mobilevit-small                                  NaN   

dataset                                                                                                       \
metric                                                                                                L2_acc   
ft_strategy               model_type          loss_function backbone                                           
full_fine_tuning_50epochs fully convolutional TRADES_v2     convnext_tiny.fb_in1k                      0.850   
                          fully attention     TRADES_v2     vit_base_patch16_clip_224.laion2b_ft_in1k  0.820   
                          fully convolutional TRADES_v2     convnext_tiny.fb_in22k                     0.842   
                                                            robust_convnext_tiny                       0.844   
                                                            edgenext_small.usi_in1k                    0.811   
...                                                                                                      ...   
full_fine_tuning_5epochs  hybrid              CLASSIC_AT    mobilevit-small                              NaN   
linearprobe_50epochs      fully convolutional CLASSIC_AT    edgenext_small.usi_in1k                      NaN   
                                                            efficientnet-b0                              NaN   
                                                            mobilenetv3_large_100.ra_in1k                NaN   
                          hybrid              TRADES_v2     mobilevit-small                              NaN   

dataset                                                                                                          \
metric                                                                                                 Linf_acc   
ft_strategy               model_type          loss_function backbone                                              
full_fine_tuning_50epochs fully convolutional TRADES_v2     convnext_tiny.fb_in1k                      0.778000   
                          fully attention     TRADES_v2     vit_base_patch16_clip_224.laion2b_ft_in1k  0.774000   
                          fully convolutional TRADES_v2     convnext_tiny.fb_in22k                     0.763000   
                                                            robust_convnext_tiny                       0.899000   
 

In [ ]:
# Flatten the columns
df_sorted.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in df_sorted.columns]
df_reset = df_sorted.reset_index()
sub_df = df_reset[["ft_strategy", "model_type", "backbone", "loss_function", "TOTAL_score_sum"]]
sub_df.to_csv("./global_dataset.csv")

In [5]:
import pandas as pd
import plotly.express as px


custom_palette = ["#E69F00", "#56B4E9", "#009E73"]  # choose a set above

# ── 1.  Rank backbones by their aggregate TOTAL_score_sum ───────────────────────
backbone_order = (
    sub_df.groupby("backbone")["TOTAL_score_sum"]
          .sum()                       # sum over the 6 settings
          .sort_values(ascending=False)
          .index                       # → ordered list of backbones
          .tolist()
)

# Make the column an ordered categorical so Plotly respects the ranking
sub_df["backbone"] = pd.Categorical(
    sub_df["backbone"],
    categories=backbone_order,
    ordered=True
)

# ── 2.  Custom colours / markers ───────────────────────────────────────────────
color_map = {
    "full_fine_tuning_50epochs": custom_palette[0],#"orange",
    "full_fine_tuning_5epochs":  custom_palette[1],#"purple",
    "linearprobe_50epochs": custom_palette[2]     #"green",
}
symbol_map = {"TRADES_v2": "star", "CLASSIC_AT": "square"}

# ── 3.  Plot ───────────────────────────────────────────────────────────────────
fig = px.scatter(
    sub_df,
    width=1500, 
    height=1000,
    color_discrete_sequence=custom_palette,
    x= "backbone",
    y="TOTAL_score_sum",
    color="ft_strategy",
    symbol="loss_function",
    color_discrete_map=color_map,
    symbol_map=symbol_map,
    hover_data=["backbone", "ft_strategy", "loss_function", "TOTAL_score_sum"],
)

fig.update_layout(
    paper_bgcolor="white",   # outside the plotting area
    plot_bgcolor="white",    # inside the plotting area

    xaxis=dict(
        showgrid=True,
        gridcolor="lightgrey",
        zeroline=False       # optional: hide the heavy axis line
    ),
    yaxis=dict(
        showgrid=True,
        gridcolor="lightgrey",
        zeroline=False
    )
)

fig.update_layout(yaxis_title="Aggregate Accuracy Score")

fig.update_layout(showlegend=True)   # keep or turn off if it clutters
fig.show()

fig.write_image("./global_plot.png", scale=3  )                # upscale for higher DPI (1 = default)



/var/folders/v7/3s0lms795672_f7_mh2x6bcr0000gn/T/ipykernel_74754/3436293082.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df["backbone"] = pd.Categorical(


In [5]:
grouped_df

,ft_strategy,model_type,loss_function,backbone,TOTAL_borda,TOTAL_nan_geom_cnt,TOTAL_nan_sum_cnt,TOTAL_rank_borda,TOTAL_rank_geom,TOTAL_rank_sum,...,stanford_cars_sum,uc-merced-land-use-dataset_L1_acc,uc-merced-land-use-dataset_L2_acc,uc-merced-land-use-dataset_Linf_acc,uc-merced-land-use-dataset_borda,uc-merced-land-use-dataset_clean_acc,uc-merced-land-use-dataset_common_acc,uc-merced-land-use-dataset_geom,uc-merced-land-use-dataset_model_size,uc-merced-land-use-dataset_sum
0,full_fine_tuning_50epochs,fully convolutional,TRADES_v2,convnext_tiny.fb_in22k_ft_in1k,6082.0,0,0,1.0,1.0,1.0,...,2.029667,0.784127,0.752381,0.511905,1023.0,0.952381,0.803175,0.231012,1,3.803968
1,full_fine_tuning_50epochs,fully convolutional,TRADES_v2,convnext_tiny.fb_in1k,5693.0,0,0,6.0,6.0,2.0,...,2.035333,0.701587,0.761905,0.580952,983.0,0.895238,0.717460,0.199462,1,3.657143
2,full_fine_tuning_50epochs,fully attention,TRADES_v2,vit_base_patch16_clip_224.laion2b_ft_in1k,5896.0,0,0,4.0,7.0,3.0,...,2.268000,0.585714,0.809524,0.733333,1018.0,0.952381,0.777778,0.257563,2,3.858730
3,full_fine_tuning_50epochs,fully convolutional,TRADES_v2,convnext_tiny.fb_in22k,5903.0,0,0,3.0,8.0,4.0,...,0.220667,0.580952,0.438095,0.228571,969.0,0.952381,0.792857,0.043928,1,2.992857
4,full_fine_tuning_50epochs,fully convolutional,TRADES_v2,robust_convnext_tiny,5696.0,0,0,5.0,4.0,5.0,...,0.607667,0.504762,0.835714,0.859524,1064.0,0.959524,0.822222,0.286054,1,3.981746
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,linearprobe_50epochs,hybrid,CLASSIC_AT,coatnet_2_rw_224.sw_in12k_ft_in1k,0.0,6,6,235.0,172.0,235.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN
236,linearprobe_50epochs,hybrid,CLASSIC_AT,coatnet_0_rw_224.sw_in1k,0.0,6,6,235.0,172.0,235.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN
237,linearprobe_50epochs,hybrid,TRADES_v2,coatnet_0_rw_224.sw_in1k,0.0,6,6,235.0,172.0,235.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN
238,linearprobe_50epochs,hybrid,TRADES_v2,coatnet_2_rw_224.sw_in12k,0.0,6,6,235.0,172.0,235.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN


In [3]:
import pandas as pd
from scipy.stats import spearmanr, pearsonr
from process_database import process_grouped_df, process_rankings,global_grouped_dataset


size = 1 
grouped_df = global_grouped_dataset(df,)
grouped_df = process_rankings(grouped_df)
# Flatten the columns
grouped_df.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in grouped_df.columns]
grouped_df = grouped_df.reset_index()
sub_df = grouped_df[["ft_strategy", "model_type", "backbone", "loss_function", "TOTAL_score_sum"]]

# ── 1.  Reshape: one row per backbone × loss_function, one column per project
pivot_df = sub_df.pivot_table(
    index=["backbone", "loss_function"],
    columns="ft_strategy",
    values="TOTAL_score_sum"
).reset_index()

# ── 2.  Prepare correlation results ───────────────────────────────────────────
project_pairs = [
    ("linearprobe_50epochs",  "full_fine_tuning_50epochs"),
    ("linearprobe_50epochs",  "full_fine_tuning_5epochs"),
    ("full_fine_tuning_5epochs", "full_fine_tuning_50epochs"),
]

correlation_results = []

for p1, p2 in project_pairs:
    if p1 in pivot_df.columns and p2 in pivot_df.columns:
        valid_rows = pivot_df[[p1, p2]].dropna()
        if len(valid_rows) >= 2:           # need ≥2 points for correlation
            pearson_corr, pearson_p   = pearsonr(valid_rows[p1], valid_rows[p2])
            spearman_corr, spearman_p = spearmanr(valid_rows[p1], valid_rows[p2])
            correlation_results.append({
                "comparison":  f"{p1}  vs  {p2}",
                "pearson":     pearson_corr,
                "pearson_p":   pearson_p,
                "spearman":    spearman_corr,
                "spearman_p":  spearman_p
            })

overall_corr_df = pd.DataFrame(correlation_results)
overall_corr_df


,comparison,pearson,pearson_p,spearman,spearman_p
0,linearprobe_50epochs vs full_fine_tuning_50e...,0.611436,1.697557e-09,0.620814,8.092712e-10
1,linearprobe_50epochs vs full_fine_tuning_5ep...,0.627018,4.890366e-10,0.595961,5.473715e-09
2,full_fine_tuning_5epochs vs full_fine_tuning...,0.434381,5.676906e-05,0.478528,7.132718e-06


In [4]:
pivot_df

ft_strategy,backbone,loss_function,full_fine_tuning_50epochs,full_fine_tuning_5epochs,linearprobe_50epochs
0,CLIP-convnext_base_w-laion2B-s13B-b82K,CLASSIC_AT,11.303238,0.948095,3.956905
1,CLIP-convnext_base_w-laion2B-s13B-b82K,TRADES_v2,11.950302,4.634952,7.543333
2,CLIP-convnext_base_w-laion_aesthetic-s13B-b82K,CLASSIC_AT,10.542286,1.013429,2.857746
3,CLIP-convnext_base_w-laion_aesthetic-s13B-b82K,TRADES_v2,13.974556,5.568825,7.351238
4,coat_tiny.in1k,CLASSIC_AT,1.593095,0.452000,0.352857
...,...,...,...,...,...
75,vit_small_patch16_224.augreg_in21k,TRADES_v2,8.256286,2.750571,2.792762
76,vit_small_patch16_224.augreg_in21k_ft_in1k,CLASSIC_AT,6.104857,1.101921,2.748937
77,vit_small_patch16_224.augreg_in21k_ft_in1k,TRADES_v2,8.685698,0.921730,3.284619
78,vit_small_patch16_224.dino,CLASSIC_AT,6.640190,1.068413,2.643746


In [ ]:
# ── Correlations by loss_function subgroup ────────────────────────────────────

loss_corrs = []
for loss in pivot_df["loss_function"].unique():
    subset = pivot_df[pivot_df["loss_function"] == loss]

    for p1, p2 in project_pairs:
        if p1 in subset.columns and p2 in subset.columns:
            valid_rows = subset[[p1, p2]].dropna()

            if len(valid_rows) >= 2:          # need ≥2 points for correlation
                pearson_corr, pearson_p   = pearsonr(valid_rows[p1], valid_rows[p2])
                spearman_corr, spearman_p = spearmanr(valid_rows[p1], valid_rows[p2])

                loss_corrs.append({
                    "loss_function": loss,
                    "comparison":   f"{p1}  vs  {p2}",
                    "pearson":      pearson_corr,
                    "pearson_p":    pearson_p,
                    "spearman":     spearman_corr,
                    "spearman_p":   spearman_p
                })

loss_corr_df = pd.DataFrame(loss_corrs)
loss_corr_df


,loss_function,comparison,pearson,pearson_p,spearman,spearman_p
0,CLASSIC_AT,linearprobe_50epochs vs full_fine_tuning_50e...,0.163357,3.138453e-01,0.221609,1.693507e-01
1,CLASSIC_AT,linearprobe_50epochs vs full_fine_tuning_5ep...,0.651009,5.402323e-06,0.642030,7.991351e-06
2,CLASSIC_AT,full_fine_tuning_5epochs vs full_fine_tuning...,0.246936,1.245046e-01,0.216329,1.800056e-01
3,TRADES_v2,linearprobe_50epochs vs full_fine_tuning_50e...,0.597639,4.663878e-05,0.592502,5.625584e-05
4,TRADES_v2,linearprobe_50epochs vs full_fine_tuning_5ep...,0.742497,4.141726e-08,0.783176,2.330621e-09
5,TRADES_v2,full_fine_tuning_5epochs vs full_fine_tuning...,0.386678,1.371146e-02,0.394095,1.186297e-02


In [ ]:
import pandas as pd
from scipy.stats import fisher_exact
from process_database import process_grouped_df, process_rankings
from load_results import load_result_dataset

# Collector for odds ratios and p-values
results = []

for (name, pn1, pn2) in [

    ('Full fine-tuning (50 epochs)','full_fine_tuning_50epochs_edge_paper_final2', 'full_fine_tuning_50epochs_paper_final2'),
    ('Full fine-tuning (5 epochs)', 'full_fine_tuning_5epochs_edge_article1', 'full_fine_tuning_5epochs_article1'),
    ('Linear probing (50 epochs)', 'linearprobe_50epochs_edge_paper_final2', 'linearprobe_50epochs_paper_final2') ]:
    
    final_data = load_result_dataset(pn1, pn2)

    for size_id, size in [(0, 'small'), (1, 'medium'), (2, 'large')]:

        grouped_df = process_grouped_df(final_data, size_id)
        grouped_df = process_rankings(grouped_df)

        odds_df = grouped_df['TOTAL'].reset_index()

        # Unique ID for observation
        index_cols = ['backbone', 'pre_training_strategy', 'model_type']
        score_col = "score_sum"

        pivot = odds_df.pivot_table(
            index=index_cols,
            columns="loss_function",
            values=score_col
        ).dropna()

        # Pairwise TRADES vs CLASSIC
        pivot["winner"] = pivot.apply(
            lambda row: "TRADES" if row["TRADES_v2"] > row["CLASSIC_AT"]
            else "CLASSIC" if row["TRADES_v2"] < row["CLASSIC_AT"]
            else "TIE",
            axis=1
        )

        wins = pivot[pivot["winner"] != "TIE"]
        trades_wins = (wins["winner"] == "TRADES").sum()
        classic_wins = (wins["winner"] == "CLASSIC").sum()

        table = [[trades_wins, classic_wins], [classic_wins, trades_wins]]
        odds_ratio, p_value = fisher_exact(table)

        # Collect result
        results.append({
            "protocol": name,
            "size": size,
            "trades_wins": trades_wins,
            "classic_wins": classic_wins,
            "odds_ratio": odds_ratio,
            "p_value": p_value
        })

# Convert to DataFrame
results_df = pd.DataFrame(results)

../results/full_fine_tuning_50epochs_paper_final2/CLIP-convnext_base_w-laion_aesthetic-s13B-b82K_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_paper_final2/CLIP-convnext_base_w-laion2B-s13B-b82K_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_paper_final2/deit_small_patch16_224.fb_in1k_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_paper_final2/robust_resnet50_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_paper_final2/vit_small_patch16_224.augreg_in21k_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_paper_final2/convnext_base.fb_in1k_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_paper_final2/resnet50.a1_in1k_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_paper_final2/robust_vit_base_patch16_224_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_

In [23]:
import plotly.express as px
import pandas as pd

# Add sample size column
results_df["sample_size"] = results_df["trades_wins"] + results_df["classic_wins"]

# Format text label with odds ratio, p-value, and sample size
results_df["text_label"] = results_df.apply(
    lambda row: (
        f'{int(row["odds_ratio"])}' if row["odds_ratio"] == int(row["odds_ratio"]) 
        else f'{row["odds_ratio"]:.2f}'
    ) + f'<br>(p={row["p_value"]:.3f})<br>n={row["sample_size"]}',
    axis=1
)
# Plotly bar chart
fig = px.bar(
    results_df,
    x="size",
    y="odds_ratio",
    color="protocol",
    barmode="group",
    text="text_label",
    labels={"size": "Model Size", "odds_ratio": "Odds Ratio"},
)

# Update traces for text styling
fig.update_traces(
    textposition="outside",
    textangle=-90,
    textfont=dict(size=10),
    cliponaxis=False
)

fig.update_layout(
            margin=dict(l=0, r=0, t=0, b=0),  # remove all outer margins
            width=450,    # in pixels (~2.5 inches at 96 DPI)
            height=400,    # in pixels (~1.875 inches)
            yaxis_title="Odds Ratio",
            xaxis_title="Model Size",            
            # White background
            plot_bgcolor="white",
            paper_bgcolor="white",
            
            # Light grey grid
            xaxis=dict(showgrid=False, gridcolor="lightgrey"),
            yaxis=dict(showgrid=True, gridcolor="lightgrey"),
            uniformtext_minsize=8,
            uniformtext_mode='show',
            
            # Legend inside figure
            legend=dict(
                font=dict(size=12),
                yanchor="top",
                xanchor="left",   # anchor point for x position
                x=0.05,           # move legend to the left side
                y=1.15,           # optional: near the top
                bgcolor="rgba(255,255,255,1)",
                bordercolor="lightgrey",
                borderwidth=1
            )
        )


fig.show()
fig.write_image("./paper_figures/oddsratio_TRADES_outperforms.png", scale=3  )
